## Abstract

@Rijal2025 recently applied attention mechanisms to the problem of mapping genotype to phenotype. We noted their architecture omitted standard Transformer components, prompting us to test if including these elements could enhance performance on their multi-environment yeast dataset. Our analysis revealed that incorporating standard Transformer elements substantially improves predictive accuracy for this task.

----

:::{.callout-note title="AI usage disclosure" collapse="true"}
This is a placeholder for the AI usage disclosure. Once all authors sign the AI code form on AirTable, SlackBot will message you an AI disclosure that you should place here.
:::

# Introduction

The recent preprint by @Rijal2025 introduces an application of attention mechanisms for inferring genotype-phenotype maps, particularly focusing on capturing complex epistatic interactions. They showed that their attention-based model outperformed linear and linear + pairwise models. Overall, their work sparked considerable interest and discussion within our journal club.

While appreciating the novelty of applying attention in this domain, we noted that the specific architecture employed is relatively minimal. It utilizes stacked attention layers but omits several components commonly found in the original transformer model [@Vaswani2017], such as skip connections, layer normalization, or feed-forward blocks. We found these omissions interesting, since the transformer was the first architecture to fully leverage the power of attention mechanisms, and did so to great success in many distinct domains.

This led us to a very straightforward inquiry: Could the performance of the attention-based genotype-phenotype model proposed by @Rijal2025 be improved by replacing it with a standard transformer architecture?"

## The dataset

The experimental data used in @Rijal2025 comes from the work of @Ba2022, who performed a large-scale quantitative trait locus (QTL) study in yeast. In short, they measured the growth rates of ~100,000 yeast segregants across 18 conditions and for ~40,000 loci, creating a massive dataset suitable for mapping genotype to phenotype.

Due to extensive linkage disequilibrium (LD), the loci in the dataset are highly correlated with each other. To create a set of independent loci, @Rijal2025 a defined a set of loci such that the correlation between the SNPs present at any pair of loci is less than 94%, resulting in a set of 1164 "independent" loci.

Unfortunately, they didn't provide this set of loci, nor the genotypic and phenotypic data used for training, so we located the [raw data](https://datadryad.org/dataset/doi:10.5061/dryad.1rn8pk0vd) that @Ba2022 originally uploaded alongside their study, then used [this notebook](https://github.com/Emergent-Behaviors-in-Biology/GenoPhenoMapAttention/blob/main/obtain_independent_loci.ipynb) uploaded by @Rijal2025 to recapitulate the 1164 loci. To save everyone else the trouble, we uploaded the train, test, and validation datasets we're *pretty sure* @Rijal2025 used in their study.

::: {.callout-note}
Roughly 2% of the phenotypic data are undefined (NaN).
:::

You can find the data here:

TODO

This data marks the start of our analysis, so we begin by downloading it:

In [4]:
import subprocess

# TODO 
download_command_string = "aws s3 sync s3://2025-attention-is-almost-all-you-need/inputs/ inputs/"
completed_process = subprocess.run(download_command_string.split(" "))
assert completed_process.returncode == 0

ModuleNotFoundError: No module named 'boto3'